In [1]:
from snowflake.snowpark import Session
import warnings; warnings.simplefilter('ignore')
import configparser

In [2]:
# Fosfor ML
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [3]:
#Import all snowflake connection details from template variables.
db_user = 'ASHUTOSH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'RETAIL_DB'
db_role = 'ASHUTOSH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'CPG_SAP_OTIF_SCHEMA'

In [4]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [5]:
df = session1.table("ENRICHED_ORDER_SHIP")

In [6]:
df = df.to_pandas()

In [7]:
df.shape

(966997, 46)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 46 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   CUSTOMER_PURCHASE_ORDER_DATE          966997 non-null  object 
 1   ORDER_ENTRY_DATE                      966997 non-null  object 
 2   ORDER_REQUIRED_DELIVERY_DATE          966997 non-null  object 
 3   ORDER_CUSTOMER_REQUEST_DELIVERY_DATE  966997 non-null  object 
 4   ORDER_PROMISE_DATE                    966997 non-null  object 
 5   ORDER_FIRST_SHIPMENT_DATE             966997 non-null  object 
 6   SHIPMENT_ACTUAL_START_DATE            966997 non-null  object 
 7   ORDER_LINECANCEL_DATE                 966997 non-null  object 
 8   ITEM_OTQF                             758063 non-null  object 
 9   ITEM_OTQF_FLAG                        966997 non-null  object 
 10  ITEM_OTQF_DAYS                        847138 non-null  float64
 11  